# Пробник GeoPy

## Импортируем библиотеку:

In [5]:
import geopy

import pickle as pk
import pandas as pd

import requests
from bs4 import BeautifulSoup as BS

import time

## Работа с пиклом по достопримечательностям

In [40]:
with open("./parser_data.pkl", "rb") as file:
    data = pd.read_pickle(file)

In [41]:
data.head()

,Href,Title,Content
0,https://wikiway.com/egypt/giza/egipetskie-pira...,Египетские пирамиды,"Египетские пирамиды, расположенные недалеко от..."
1,https://wikiway.com/italia/rim/kolizey/,Колизей,Колизей – один из самых узнаваемых архитектурн...
2,https://wikiway.com/russia/moskva/krasnaya-plo...,Красная площадь,Красная площадь – основной символ и наиболее и...
3,https://wikiway.com/izrail/mertvoe-more/,Мертвое море,"Мертвое море — это уникальный подарок природы,..."
4,https://wikiway.com/algeria/sakhara/,Сахара,"Сахара — крупнейшая пустыня, расположенная в С..."


In [42]:
# # Получаем массив строк [страна, <город>, достопримечательность]
# address = []
# for line in data.Href:
#     line_ar = [i for i in line.split("/")]
    
#     for i in range(len(line_ar)):
        
#         if ".com" in line_ar[i]:
#             new_address = []
            
#             for j in range(i+1, len(line_ar) - 1):
#                 new_address.append(line_ar[j])
            
#             address.append(new_address)
#             break            

### делаем словарь и запихиваем его в пикл

In [43]:
# # составление словаря {Страна : {"Cities" : [города], :"Sight" [достопримечательности (если нет города)]}}
# address_dict = {}
# for i in address:
#     key = i[0]
#     if key not in address_dict.keys():
#         address_dict[key] = {"Cities":[],"Sight":[]}
#     value = i[1]
#     if len(i) == 3:
#         if value not in address_dict[key]["Cities"]:
#             address_dict[key]["Cities"].append(value)
#     else:
#         if value not in address_dict[key]["Sight"]:
#             address_dict[key]["Sight"].append(value)

In [44]:
# вид: {Страна : {"Cities" : [], :"Sight" []}}

In [45]:
# # Изменение названий на те, что лежат в data.title
# for key in address_dict.keys():
#     for name_i in range(len(address_dict[key]["Sight"])):
#         for line_i in data.index:
#             if address_dict[key]["Sight"][name_i] in data.loc[line_i]["Href"]:
#                 address_dict[key]["Sight"][name_i] = data.loc[line_i]["Title"]

In [46]:
#address_dict

In [47]:
# # Перевод всех названий на официальные названия на английском с помощью WIKIpedia
# for key in address_dict:
#     for ind in range(len(address_dict[key]['Sight'])):
#         url = "https://ru.wikipedia.org/wiki/{}".format(address_dict[key]['Sight'][ind].replace(" ","_"))
#         x = BS(requests.get(url).text, "html.parser")
#         x = x.find_all("div", {"class": "body"})
#         Href = ""
#         for y in x:
#             y = y.find_all("a")
#             for z in y:
#                 if z.text.strip() != "English":
#                     continue
#                 else:
#                     Href = z.get("href")
#                     break
#         if Href == "":
#             continue
#         line = [i for i in Href.split("/")]
#         address_dict[key]['Sight'][ind] = line.pop().strip().replace("_", " ")

In [48]:
#address_dict

In [49]:
# for key in address_dict.keys():
#     x = address_dict[key]
#     for i in x.keys():
#         for j in range(len(x[i])):
#             x[i][j] = str(x[i][j]).replace("%27", "'")
# x[i][j] = str(x[i][j]).replace("(disambiguation)","")
# x[i][j] = str(x[i][j]).replace("Pripyat", "Pripyat town").strip()

In [50]:
# plk_dict = {}
# for key in address_dict.keys():
#     x = address_dict[key]
#     for i in x.keys():
#         for j in x[i]:
#             print(j, type(j))

In [51]:
# with open("./dict.pkl", "wb") as file:
#     pk.dump(address_dict, file)

----

In [52]:
# addr = []
# for i in address:
#     line = []
#     line.append(i[0])
#     if len(i) == 3:
#         line.append(i[1])
#     else:
#         line.append(None)
#     addr.append(line)
    
# data = data.join(pd.DataFrame(addr,columns=["Country", "City"]))

In [62]:
def translate(data_):
    Off_name= []
    for key in data_:
        l = []

        url = "https://ru.wikipedia.org/wiki/{}".format(key.strip().replace(" ","_"))
    
        x = BS(requests.get(url).text, "html.parser")
        x = x.find_all("div", {"class": "body"})
    
        Href = ""
    
        for y in x:
            y = y.find_all("a")
            for z in y:
                if z.text.strip() != "English":
                    continue
                else:
                    Href = z.get("href")
                    break
    
        if Href == "":
            n = ""
        else:
            line = [i for i in Href.split("/")]
            n = line.pop().strip().replace("_", " ")
            n = str(n).replace("%27", "'").replace("(disambiguation)","").strip()
            n = n.replace("%C3%AD", "i")
        l = [key, n]
        Off_name.append(l)
    return Off_name

In [54]:
# data = data.merge(pd.DataFrame(Off_name, columns = ["Title","Off_name"]),suffixes=("",""),on=['Title'])

In [55]:
# data = data[["Title", "Off_name", "Content", "Href"]]

In [56]:
# data.head()

## продолжаем geopy

### получаем координаты для каждой из достопримечательностей

In [17]:
geo = geopy.geocoders.Yandex
geolocator = geo(user_agent="Sight_addr")

In [85]:
def lat_lon_addr(data_):
    latitude_longitude = []
    i = 0
    wtf = []
    for name in data_:
        time.sleep(1)
        
        i += 1
        print(i, name)
    
        try:
            location = geolocator.geocode(name)
        except:
            wtf.append(name)
            continue
        
        if location == None:
            wtf.append(name)
            continue
    
        line = [name, location.address, location.latitude, location.longitude]
        latitude_longitude.append(line)
    return {"data_completed": latitude_longitude, "data_error": wtf}

In [78]:
data_tittle_off = data.Title

In [84]:
data_tittle_off = translate(data_tittle_off)

In [93]:
data_tittle_off_translate = []
for i in data_tittle_off:
    if i[1] != "":
        data_tittle_off_translate.append(i[1])
    else:
        data_tittle_off_translate.append(i[0])

In [94]:
len(data_tittle_off_translate)

100

In [95]:
data_dict = lat_lon_addr(data_tittle_off_translate)

1 Egyptian pyramids
2 Colosseum
3 Red Square
4 Dead Sea
5 Sahara
6 Tibet
7 Eiffel Tower
8 Hollywood
9 Canary Islands
10 Manhattan
11 Остров Крит
12 Reichstag
13 Peterhof Palace
14 Диснейленд в Париже
15 Louvre
16 Moscow Kremlin
17 Normandy
18 Pamukkale
19 Pripyat
20 Provence
21 Sagrada Familia
22 Sunny Beach
23 Статуя Свободы (Statue of Liberty)
24 Taj Mahal
25 Big Ben
26 Great Wall of China
27 Гренландия
28 Mal%C3%A1 Strana
29 Lake Baikal
30 Notre-Dame de Paris
31 Fort Boyard (fortification)
32 Goa
33 Mount Everest
34 Machu Picchu
35 Bali
36 Saint Basil's Cathedral
37 Cappadocia
38 Niagara Falls
39 Petra
40 Western Wall
41 Stonehenge
42 Transylvania
43 Московский государственный университет имени М.В. Ломоносова (МГУ)
44 Sphinx
45 Mount Elbrus
46 Easter Island
47 Palmyra
48 Great Pyramid of Giza
49 Poklonnaya Hill
50 Bolshoi Theatre
51 «Брестская крепость»
52 Brooklyn
53 Tretyakov Gallery
54 Zanzibar Archipelago
55 Josefov
56 Moskva River
57 Озеро Хевиз
58 Santorini
59 Leaning Tower o

In [96]:
lla = data_dict["data_completed"]
wtf = data_dict["data_error"]

In [98]:
wtf

['Leaning Tower of Pisa',
 'Cathedral of Christ the Saviour',
 'Neuschwanstein Castle',
 'Phuket Province',
 'Plitvice Lakes National Park']

In [99]:
lla

[['Egyptian pyramids',
  'комплекс пирамид Эль-Гиза, район Старый Каир, Каир, Египет',
  29.975773,
  31.129975],
 ['Colosseum', 'Колоссеум, Квинсленд, Австралия', -24.388286, 151.612192],
 ['Red Square', 'Красная площадь, Москва, Россия', 55.753564, 37.621085],
 ['Dead Sea', 'Мёртвое море', 31.534797, 35.490577],
 ['Sahara', 'Западная Сахара', 24.606218, -13.289631],
 ['Tibet', 'Тибетский автономный район, Китай', 31.518361, 88.408534],
 ['Eiffel Tower', 'Тауэр, Манстер, Ирландия', 51.925175, -8.607962],
 ['Hollywood',
  'город Голливуд, Броуард-Каунти, штат Флорида, Соединённые Штаты Америки',
  26.041504,
  -80.192578],
 ['Canary Islands', 'Острова Кука', -21.240368, -159.779656],
 ['Manhattan', 'остров Манхэттен, Пекин, Китай', 39.809568, 116.278838],
 ['Остров Крит', 'остров Крит, Греция', 35.230528, 24.844921],
 ['Reichstag',
  'Im Reichstag, Бохум, Нордрхайн-Вестфален, Германия',
  51.409615,
  7.159717],
 ['Peterhof Palace',
  'Дворцовая площадь, Петергоф, Санкт-Петербург, Росс

In [18]:
location = geolocator.geocode("Московский государственный университет имени М.В. Ломоносова (МГУ)")
print(location)
location

посёлок Московский, муниципальное образование Город Екатеринбург, Свердловская область, Россия


Location(посёлок Московский, муниципальное образование Город Екатеринбург, Свердловская область, Россия, (56.820553, 60.496674, 0.0))

In [1]:
class PrefixTree:
    def __init__(self):
        self.root = [{}]

    def add(self, string, node_item=None, periodicity=1):
        # добавить строку
        wrk_dict = self.root

        if self.check(string):
            for i in string:
                wrk_dict = wrk_dict[0][i]
            wrk_dict[1] += 1

        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                wrk_dict[0][i] = [{}]
                wrk_dict = wrk_dict[0][i]

        wrk_dict.append(periodicity)
        wrk_dict.append(node_item)

    def check(self, string):
        # проверить наличие строки
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        if len(wrk_dict) != 1:
            return True
        return False

    def check_part(self, string):
        # проверить наличие строки как префикса
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        return True

    def get_n_sag(self, string, n):
        # TODO возвращает массив
        top = []
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                break

        if len(wrk_dict) != 1:
            top.append(string, wrk_dict[1], wrk_dict[2])

        def rec(t, wrk_d, st):
            for i in wrk_d[0].keys():
                st += i
                print(i, wrk_d[0].keys())
                if len(wrk_dict[0][i]) == 3:
                    t.append(st, wrk_d[1], wrk_d[2])
                if len(wrk_d[0][i][0]) != 0:
                    rec(t, wrk_d[0][i], st)

        rec(top, wrk_dict, string)
        return top.sort(key=lambda x: x[1], reverse=True)[:n]


def init_prefix_tree(filename):
    prefix_tree = PrefixTree()
    ar_to_el = []

    with open("./"+filename, "r") as file:
        for line in file:
            l = [i for i in line.split("\t")]
            ar_to_el.append(l)

    for line in ar_to_el:
        prefix_tree.add(line[0], line[1], line[2])

    return prefix_tree


def return_sudgest(string):
    top = prefix_tree.get_n_sag(string, 10)
    for i in range(len(top)):
        top[i] = [top[i][0], top[i][2]]

    if request.method == "GET":
        return jsonify(top)


def hello():
    return "Welcome on this server\nPlease go to /get_sudgest/"
    # TODO должна возвращатьс инструкция по работе с сервером


prefix_tree = init_prefix_tree("For_tree.txt")
